In [4]:
import os
import numpy as np
from getpass import getpass

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#LLMS
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

#Libraries for word and vector embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [6]:
data_path = "Resume_data"

files = os.listdir(data_path)

In [12]:
def load_data(file):
  loader = UnstructuredPDFLoader(file)
  return loader.load() 

In [13]:
resumes = {}

for i in files:
  resumes[i] = load_data("Resume_data/{}".format(i))

[nltk_data] Downloading package punkt to /Users/vsvsvarma/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vsvsvarma/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [15]:
print("Enter OpenAI private key:")
OPENAI_PRIVATE_KEY = getpass()

Enter OpenAI private key:


In [17]:
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_PRIVATE_KEY)


In [18]:
data = [ i[0] for i in list(resumes.values())]

In [21]:
db = Chroma.from_documents(data, embeddings)

In [22]:
query = input()

In [24]:
search_results = db.similarity_search(query, k = 2)
search_results

[Document(page_content="Sanskriti Garg San Francisco, CA -Email me on Indeed: http://www.indeed.com/r/Sanskriti-Garg/07feaa041e30e827\n\nWork Experience\n\nBrand Manager Marico India - Mumbai, Maharashtra August 2019 to January 2022\n\nHeaded strategic business planning and go-to-market strategy for 3 brands with $35M revenue. Managed a Brand Executive. • Led cross-functional team to reposition a declining brand; Achieved 90 bps penetration gain within first 12 months of launch • Created go-to-market strategy for pan-India product relaunch and organized product launch briefs for key leadership stakeholders and 100+ sales team • Implemented A/B testing of 30+ digital creatives to optimize digital marketing strategy and effective deployment across Meta, YT • Researched competitors' UX/UI, gathered UX feedback from 30+ users; Provided feedback to design team to create brand UX/UI • Spearheaded cross-functional collaboration of 10 from product, design and external agencies to develop low-c

In [25]:
universities = "Cornell University"

In [26]:
prompt = "I want you to act as an education counseller and recommend potential carreer paths to student who's graduating from his bachelor's degree and planning on moving to USA for masters. Based on the resumes below recommend student some of the hard skills that he needs to learn & include how {} can help him achieve these skills and advance in his career.".format(universities) + query

In [27]:
llm = OpenAI(temperature = 0, openai_api_key= OPENAI_PRIVATE_KEY, max_tokens= 512)
chain = load_qa_chain(llm,chain_type = 'stuff')

In [28]:
llm = OpenAI(temperature = 0, openai_api_key= OPENAI_PRIVATE_KEY, max_tokens= 512)
chain = load_qa_chain(llm,chain_type = 'stuff')

In [29]:
response = chain.run(input_documents = search_results, question = prompt)
response

'\n\nBased on the resumes of Sanskriti Garg and Rudra Trivedi, it appears that both have a strong background in marketing and business. For Sanskriti, she has experience in brand management, product management, and shopper marketing. For Rudra, he has experience in business analysis, operations, and marketing communications.\n\nTo advance in their careers, both Sanskriti and Rudra should consider learning more about digital marketing, data analytics, and project management. Cornell University offers a range of courses in these areas, such as Digital Marketing Strategies, Data Analytics for Business, and Project Management for Business. These courses can help them develop the skills they need to succeed in their respective fields.'